## **_Backtesting Workspace_**

**_Area to backtest strategies._**

## **_Libraries_**

In [2]:
# Backtesting
import vectorbt as vbt

# Data
import yfinance as yf
import pandas as pd
import numpy as np

# Time
import datetime as dt

# Plotting
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
def Data(symbols, interval, start, end):
    """
    ## Data

    ### Description:
    This function collects close data from Yahoo Finance
    in a structure that agrees with the VectorBT Ind
    Factory function.

    ### Args:
    - symbols (List): List of symbol's data to collect
    - period (Integer): Number of most recent days to data to collect
    - interval (String): Timeframe of data to collect

    ### Returns:
    - price (Dataframe): VectorBT agreaeble dataframe of price data
    """

    # Downloading close data
    price = vbt.YFData.download(symbols=symbols,
                                missing_index="drop",
                                start=start,
                                end=end,
                                interval=interval).get("Close")
    
    # Returning the close prices
    return price

## **_BBands Functions_**

In [4]:
def BBands_Indicator(price, ma_window, std_size):
    """
    ## BBands Indicator

    ### Description:
    A basic template of a BBands indicator.

    ### Args:
    - price (Dataframe): Pandas dataframe of close data
    - ma_window (Int): An integer for the moving average window
    - std_size (Int): An integer for the coefficient of the standard deviation

    ### Returns:
    - signals (Array): Numpy array of integer signals representing quantities to buy
    """

    signals = pd.DataFrame(index=price.index, columns=price.columns)

    # Looping through each column
    for column in price.columns:

        # Calculating the moving average and standard deviation
        moving_avg = price[column].rolling(window=ma_window).mean()
        standard_dev = price[column].rolling(window=ma_window).std()

        # Calculating the bands
        upper_band = moving_avg + (std_size * standard_dev)
        lower_band = moving_avg - (std_size * standard_dev)

        # Looping through each value of each column
        for idx, val in enumerate(price[column]):
            if val > upper_band.iloc[idx]:
                signals.at[idx, column] = -1
            elif val < lower_band.iloc[idx]:
                signals.at[idx, column] = 1
            else:
                signals.at[idx, column] = 0
    
    # Converting the signals to an array
    return np.array(signals.dropna())

In [5]:
def BBands_Backtest(price, ma_window, std_size, vbt_ind, cash):
    """
    ## BBands Backtest

    ### Description:
    A basic template to backtest a strategy

    ### Args:
    - price (Dataframe): Pandas dataframe of close data
    - ma_window (List): List of moving average values
    - std_size (List): List of standard deviation values
    - vbt_ind (Vbt Object): VectorBT object wrapping your custom indicator
    - cash (Int): An integer for starting cash

    ### Returns:
    - None
    """

    # Finding signals
    signals = vbt_ind.run(Price=price,
                          Moving_Average_Win=ma_window,
                          Standard_Deviation_Mag=std_size,
                          param_product=True)
    
    # Calcualting performance
    portfolio = vbt.Portfolio.from_orders(price,
                                          init_cash=cash,
                                          size=signals.Output.astype(int),
                                          size_type="Amount",
                                          freq="D")
    
    return signals, portfolio

In [6]:
BBands_Vbt_Indicator = vbt.IndicatorFactory(
        class_name="BBands Strategy",                                 
        short_name="BBands",                                          
        input_names=["Price"],                                        
        param_names=["Moving_Average_Win", "Standard_Deviation_Mag"], 
        output_names=["Output"]                                       
        ).from_apply_func(BBands_Indicator,                           
                          Moving_Average_Win=21,                      
                          Standard_Deviation_Mag=1,                   
                          keep_pd=True)                               

In [7]:
def BBands_Plotting(signals, portfolio, set):
    """
    ## BBands Plotting

    ### Note:
    Must be changed to suit specific strategy

    ### Decription:
    A function to plot the trades of a specific backtest and position status throughout.
    Also plots indicators of chosen strategy.

    ### Args:
    - portfolio (VectorBT Portfolio object): Portfolio of specific backtest
    - signals (Object): Object of various dataframes of data and signals derived from said data
    - set (Tuple): A tuple of parameters of the backtest you want to plot

    ### Returns:
    - None
    """

    # Calculating the indicators
    data = pd.DataFrame()
    data["Price"] = signals.Price[set]
    data["Moving Avg"] = data["Price"].rolling(set[0]).mean()
    data["Standard Dev"] = data["Price"].rolling(set[0]).std()
    data["Upper Band"] = data["Moving Avg"] + (set[1] * data["Standard Dev"])
    data["Lower Band"] = data["Moving Avg"] - (set[1] * data["Standard Dev"])

    # Gathering trades
    trades = portfolio.orders.records_readable[portfolio.orders.records_readable["Column"]==set]
    trades["Colour"] = trades["Side"].apply(lambda side: "green" if side == "Buy" else "red")
    trades["Marker"] = trades["Side"].apply(lambda side: "triangle-up" if side == "Buy" else "triangle-down")

    # Calculating the position 
    pos_count_current = 0
    pos_count = []
    for i in range(len(signals.Output[set])):
        pos_count_current += signals.Output[set].iloc[i]
        pos_count.append(pos_count_current)
    pos_count = [f"Position: {pos}" for pos in pos_count]

    # Creating the figure
    fig = go.Figure()

    # Plotting the close price
    fig.add_trace(go.Scatter(x=data.index,
                             y=data["Price"],
                             mode="lines",
                             name="Price",
                             hovertext=pos_count))
    
    # Plotting the moving average
    fig.add_trace(go.Scatter(x=data.index,
                             y=data["Moving Avg"],
                             mode="lines",
                             name="Moving Average"))
    
    # Plotting the upper band
    fig.add_trace(go.Scatter(x=data.index,
                             y=data["Upper Band"],
                             mode="lines",
                             name="Upper Band"))
    
    # Plotting the lower band
    fig.add_trace(go.Scatter(x=data.index,
                             y=data["Lower Band"],
                             mode="lines",
                             name="Lower Band"))
    
    # Plotting the trades
    fig.add_trace(go.Scatter(x=trades["Timestamp"],
                             y=trades["Price"],
                             mode="markers",
                             marker=dict(color=trades["Colour"], symbol=trades["Marker"], size=10),
                             name="Buys + Sells"))
    
    # Updating the layout
    fig.update_layout(title=dict(text=f"Backtest: {set}, Profit: ${portfolio.total_profit()[set]}", font=dict(color="white")),
                      height=600,
                      paper_bgcolor="rgba(70,70,70,1)",
                      plot_bgcolor="rgba(230,230,230,1)",
                      xaxis=dict(title="Date",
                                 tickfont=dict(color="white"),
                                 titlefont=dict(color="white"),
                                 gridcolor="rgba(0,0,0,0.1)", gridwidth=2),
                      yaxis=dict(title="Price",
                                 tickfont=dict(color="rgba(230,230,230,1)"),
                                 titlefont=dict(color="rgba(230,230,230,1)"),
                                 gridcolor="rgba(0,0,0,0.1)", gridwidth=2),
                      shapes = [go.layout.Shape(type="rect",
                                                xref="paper",
                                                yref="paper",
                                                x0=0,
                                                y0=0,
                                                x1=1,
                                                y1=1,
                                                line={'width': 3, 'color': 'black'})],
                      legend=dict(font=dict(color="rgba(230,230,230,1)")))
    
    # Displaying the figure
    fig.show()

## **_BBands Workspace_**

In [9]:
# Gathering data
price = Data(["AAPL", "AMZN", "MSFT", "PLTR", "SOFI"], "1m", dt.datetime(year=2024, month=5, day=14), dt.datetime(year=2024, month=5, day=15))

# Backtesting
BBsignals, BBportfolio = BBands_Backtest(price, [14, 21, 28, 35, 42, 49], [1, 1.25, 1.5, 1.75, 2, 2.25, 2.5], BBands_Vbt_Indicator, 100000)

# Plotting
BBands_Plotting(BBsignals, BBportfolio, BBportfolio.total_return().idxmin())

c:\Users\redmo\Desktop\Trading\TradingVenv\lib\site-packages\vectorbt\data\base.py:527: UserWarning: Symbols have mismatching index. Dropping missing data points.
  data = cls.align_index(data, missing=missing_index)


## **_MA Cross-Over Functions_**

In [40]:
def MACross_Indicator(price, fast_window, slow_window):
    """
    ## Moving Average Cross-Over Indicator

    ### Description:
    A basic template of a moving average cross-over indicator.

    ### Args:
    - price (Dataframe): Pandas dataframe of close data
    - fast_window (Int): An integer for the fast moving average window
    - slow_window (Int): An integer for the slow moving average window

    ### Returns:
    - signals (Array): Numpy array of integer signals representing quantities to buy
    """

    # Initialising a dataframe of empty signal inputs
    signals = pd.DataFrame(index=price.index, columns=price.columns)

    # Looping through each column
    for column in price.columns:

        # Calculating the moving averages
        fast_moving_avg = price[column].rolling(window=fast_window).mean()
        slow_moving_avg = price[column].rolling(window=slow_window).mean()

        # Looping through each value of each column
        for idx, val in enumerate(price[column]):
            if (fast_moving_avg.iloc[idx-1] < slow_moving_avg.iloc[idx-1]) and (fast_moving_avg.iloc[idx] > slow_moving_avg.iloc[idx]):
                signals.at[idx, column] = 5
            elif (fast_moving_avg.iloc[idx-1] > slow_moving_avg.iloc[idx-1]) and (fast_moving_avg.iloc[idx] < slow_moving_avg.iloc[idx]):
                signals.at[idx, column] = -5
            else:
                signals.at[idx, column] = 0
    
    # Converting the signals to an array
    return np.array(signals.dropna())

In [41]:
def MACross_Backtest(price, fast_window, slow_window, vbt_ind, cash):
    """
    ## Moving Average Cross-Over Backtest

    ### Description:
    A basic template to backtest a moving average cross-over strategy

    ### Args:
    - price (Dataframe): Pandas dataframe of close data
    - fast_window (List): List of fast moving average values
    - slow_window (List): List of slow moving average values
    - vbt_ind (Vbt Object): VectorBT object wrapping your custom indicator
    - cash (Int): An integer for starting cash

    ### Returns:
    - None
    """

    # Finding signals
    signals = vbt_ind.run(Price=price,
                          Fast_MA_Window=fast_window,
                          Slow_MA_Window=slow_window,
                          param_product=True)
    
    # Calcualting performance
    portfolio = vbt.Portfolio.from_orders(price,
                                          init_cash=cash,
                                          size=signals.Output.astype(int),
                                          size_type="Amount",
                                          freq="D")
    
    return signals, portfolio

In [42]:
MACross_Vbt_Indicator = vbt.IndicatorFactory(
        class_name="MA-Cross Strategy",                                 
        short_name="MA-Cross",                                          
        input_names=["Price"],                                        
        param_names=["Fast_MA_Window", "Slow_MA_Window"], 
        output_names=["Output"]                                       
        ).from_apply_func(MACross_Indicator,                           
                          Fast_MA_Window=14,                      
                          Slow_MA_Window=50,                   
                          keep_pd=True)                               

In [43]:
def MACross_Plotting(signals, portfolio, set):
    """
    ## Moving Average Cross-Over Plotting

    ### Decription:
    A function to plot the trades of a specific backtest and position status throughout.
    Also plots slow and fast moving averages.

    ### Args:
    - portfolio (VectorBT Portfolio object): Portfolio of specific backtest
    - signals (Object): Object of various dataframes of data and signals derived from said data
    - set (Tuple): A tuple of parameters of the backtest you want to plot

    ### Returns:
    - None
    """

    # Calculating the indicators
    data = pd.DataFrame()
    data["Price"] = signals.Price[set]
    data["Fast_MA"] = data["Price"].rolling(set[0]).mean()
    data["Slow_MA"] = data["Price"].rolling(set[1]).mean()

    # Gathering trades
    trades = portfolio.orders.records_readable[portfolio.orders.records_readable["Column"]==set]
    trades["Colour"] = trades["Side"].apply(lambda side: "green" if side == "Buy" else "red")
    trades["Marker"] = trades["Side"].apply(lambda side: "triangle-up" if side == "Buy" else "triangle-down")

    # Calculating the position 
    pos_count_current = 0
    pos_count = []
    for i in range(len(signals.Output[set])):
        pos_count_current += signals.Output[set].iloc[i]
        pos_count.append(pos_count_current)
    pos_count = [f"Position: {pos}" for pos in pos_count]

    # Creating the figure
    fig = go.Figure()

    # Plotting the close price
    fig.add_trace(go.Scatter(x=data.index,
                             y=data["Price"],
                             mode="lines",
                             name="Price",
                             hovertext=pos_count))
    
    # Plotting the fast moving average
    fig.add_trace(go.Scatter(x=data.index,
                             y=data["Fast_MA"],
                             mode="lines",
                             name="Fast MA"))
    
    # Plotting the slow moving average
    fig.add_trace(go.Scatter(x=data.index,
                             y=data["Slow_MA"],
                             mode="lines",
                             name="Slow MA"))
    
    # Plotting the trades
    fig.add_trace(go.Scatter(x=trades["Timestamp"],
                             y=trades["Price"],
                             mode="markers",
                             marker=dict(color=trades["Colour"], symbol=trades["Marker"], size=10),
                             name="Buys + Sells"))
    
    # Updating the layout
    fig.update_layout(title=dict(text=f"Backtest: {set}, Profit: ${portfolio.total_profit()[set]}", font=dict(color="white")),
                      height=600,
                      paper_bgcolor="rgba(70,70,70,1)",
                      plot_bgcolor="rgba(230,230,230,1)",
                      xaxis=dict(title="Date",
                                 tickfont=dict(color="white"),
                                 titlefont=dict(color="white"),
                                 gridcolor="rgba(0,0,0,0.1)", gridwidth=2),
                      yaxis=dict(title="Price",
                                 tickfont=dict(color="rgba(230,230,230,1)"),
                                 titlefont=dict(color="rgba(230,230,230,1)"),
                                 gridcolor="rgba(0,0,0,0.1)", gridwidth=2),
                      shapes = [go.layout.Shape(type="rect",
                                                xref="paper",
                                                yref="paper",
                                                x0=0,
                                                y0=0,
                                                x1=1,
                                                y1=1,
                                                line={'width': 3, 'color': 'black'})],
                      legend=dict(font=dict(color="rgba(230,230,230,1)")))
    
    # Displaying the figure
    fig.show()

## **_MA Cross-Over Workspace_**

In [44]:
# Gathering data
price = Data(["AAPL", "AMZN", "PLTR"], "1m", dt.datetime(year=2024, month=2, day=8), dt.datetime(year=2024, month=2, day=9))

# Backtesting
MACsignals, MACportfolio = MACross_Backtest(price, [14, 17, 21, 27, 30, 35], [40, 50, 60, 70, 80], MACross_Vbt_Indicator, 1000)

# Plotting
MACross_Plotting(MACsignals, MACportfolio, MACportfolio.total_return().idxmax())

## **_New Strategy_**

* **_Ideas_**